In [30]:
import os
import json
from dotenv import load_dotenv
import nest_asyncio
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from agents.exceptions import InputGuardrailTripwireTriggered
from pydantic import BaseModel
import asyncio
from openai.types.responses import ResponseTextDeltaEvent


In [27]:
load_dotenv()
nest_asyncio.apply()

In [29]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

async def main():
    # Example 1: History question
    try:
        result = await Runner.run(triage_agent, "who was the first president of the united states?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

    # Example 2: General/philosophical question
    try:
        result = await Runner.run(triage_agent, "What is the meaning of life?")
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print("Guardrail blocked this input:", e)

if __name__ == "__main__":
    asyncio.run(main())

The first President of the United States was George Washington. He was elected in 1788, took office in 1789, and served two terms until 1797. Washington is often called the "Father of His Country" for his leadership during the American Revolution and for helping to shape the new nation. He also set important precedents, such as the peaceful transfer of power and serving only two terms as president.
Guardrail blocked this input: Guardrail InputGuardrail triggered tripwire


In [31]:
async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)


if __name__ == "__main__":
    asyncio.run(main())

Of course! Here are five jokes for you:

1. Why don’t skeletons fight each other?  
Because they don’t have the guts!

2. Why did the scarecrow win an award?  
Because he was outstanding in his field!

3. How does a penguin build its house?  
Igloos it together!

4. What do you call fake spaghetti?  
An impasta!

5. Why don’t eggs tell jokes?  
Because they’d crack each other up!

In [33]:
from agents import Agent, ItemHelpers, Runner, function_tool
import random

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


if __name__ == "__main__":
    asyncio.run(main())

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 8
-- Message output:
 Hello! You caught me on a funny day—I owe you 8 jokes. Ready? Here we go:

1. Why don’t skeletons fight each other? They don’t have the guts.
2. What do you call fake spaghetti? An impasta!
3. Why did the scarecrow win an award? Because he was outstanding in his field!
4. Want to hear a joke about construction? I’m still working on it.
5. Why can't you give Elsa a balloon? Because she will let it go!
6. What did the ocean say to the beach? Nothing, it just waved.
7. Why did the math book look sad? Because it had too many problems.
8. Why don’t eggs tell each other jokes? They might crack up!

Let me know if you want more laughs!
=== Run complete ===


In [39]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool()
    ],
)

async def main():
    result = await Runner.run(agent, "Which coffee shop should I go to, taking into account my preferences and the weather today in SF?")
    print(result.final_output)

In [40]:
asyncio.run(main())

Here’s the current weather in San Francisco today, **October 26, 2025** (Sunday):

- **Current Conditions**: Mostly cloudy, temperature around **59°F (15 °C)**  
- **Forecast Today**: Mostly cloudy through the evening, dipping into the upper 50s (14–15 °C), with fog setting in past midnight .

Given the cool and overcast weather, you might prefer a cozy, inviting spot—perhaps one with comfortable indoor seating, ample warmth, and maybe a light snack to go with your coffee. If you'd like to specify preferences (ambience, Wi‑Fi, pastry options, neighborhood), I can tailor suggestions even better. In the meantime, here are a few solid options based on current opening hours and ambiance:

1. **Saint Frank Coffee** (2340 Polk St & 120 Folsom St)  
   - Known for clean, precise pour-overs in bright, minimalist and calm settings—ideal if you want an artisanal coffee experience that feels relaxing yet focused ([overheresf.com](https://www.overheresf.com/coffee-spots-san-francisco/?utm_source=o

In [41]:
from typing import Any

from pydantic import BaseModel

from agents import RunContextWrapper, FunctionTool



def do_some_work(data: str) -> str:
    return "done"


class FunctionArgs(BaseModel):
    username: str
    age: int


async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")


tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)

In [ ]:
import os

HostedMCPTool(
    tool_config={
        "type": "mcp",
        "server_label": "google_calendar",
        "connector_id": "connector_googlecalendar",
        "authorization": os.environ["GOOGLE_CALENDAR_AUTHORIZATION"],
        "require_approval": "never",
    }
)